# Demonstration of Rendering a 3d Object in Kotlin

In [4]:
%use lets-plot

In [8]:
class Point(
    var x: Float = 0f,
    var y: Float = 0f,
    var z: Float = 0f
) {
    fun getPointOnScreen(cameraPoint: Point): Pair<Float, Float> {

        if (z < 0) {
            throw IllegalArgumentException("The point is behind the screen")
        }

        // The focal length is the distance between the camera and the screen.
        val focalLength = cameraPoint.z * -1

        // The distance between the point and the camera.
        val zDistance = z - cameraPoint.z

        // the x and y coordinates of the point on the screen.
        val xProjected = ((x - cameraPoint.x) * focalLength) / (focalLength + zDistance)

        val yProjected = ((y - cameraPoint.y) * focalLength) / (focalLength + zDistance)

        return Pair(xProjected, yProjected)
    }
}

class Line {
    val points: List<Point>

    constructor(startPoint: Point, endPoint: Point) {
        points = listOf(startPoint, endPoint)
    }

    fun getPointsOnScreen(cameraPoint: Point): List<Pair<Float, Float>> {
        return points.map { it.getPointOnScreen(cameraPoint) }
    }

    fun getGeomPath(cameraPoint: Point): geomPath {
        val pointsOnScreen = getPointsOnScreen(cameraPoint)
        return geomPath(
            mapOf(
                "x" to pointsOnScreen.map { it.first },
                "y" to pointsOnScreen.map { it.second })
        )
    }
}

class Cube {
    val points: List<Point>
    val lines: List<Line>

    constructor(centerPoint: Point, size: Float) {
        points = listOf(
            Point(centerPoint.x + size, centerPoint.y - size, centerPoint.z),
            Point(centerPoint.x + size, centerPoint.y + size, centerPoint.z),
            Point(centerPoint.x - size, centerPoint.y + size, centerPoint.z),
            Point(centerPoint.x - size, centerPoint.y - size, centerPoint.z),
            Point(centerPoint.x + size, centerPoint.y - size, centerPoint.z + size),
            Point(centerPoint.x + size, centerPoint.y + size, centerPoint.z + size),
            Point(centerPoint.x - size, centerPoint.y + size, centerPoint.z + size),
            Point(centerPoint.x - size, centerPoint.y - size, centerPoint.z + size),
        )
        lines = listOf(
            Line(points[0], points[1]),
            Line(points[1], points[2]),
            Line(points[2], points[3]),
            Line(points[3], points[0]),
            Line(points[4], points[5]),
            Line(points[5], points[6]),
            Line(points[6], points[7]),
            Line(points[7], points[4]),
            Line(points[0], points[4]),
            Line(points[1], points[5]),
            Line(points[2], points[6]),
            Line(points[3], points[7]),
        )
    }

    fun getPointsOnScreen(cameraPoint: Point): List<Pair<Float, Float>> {
        return points.map { it.getPointOnScreen(cameraPoint) }
    }

    fun getGeomPaths(cameraPoint: Point): List<geomPath> {
        return lines.map { it.getGeomPath(cameraPoint) }
    }
}

// Defines the point of the camera in 3D space.
// The camera has to be in front if the screen whitch means that the z coordinate has to be negative.
val camera = Point(-2f, -2f, -1f)


// Create a square in the 3D space
val cube1 = Cube(Point(0f, 0f, 1f), 1f)
val cube2 = Cube(Point(0f, 0f, 3f), 1f)


// Create a 2d plot that represents the screen with x and y axis
val plot = letsPlot() { x = "x"; y = "y" }

val cubeGeomPaths = cube1.getGeomPaths(camera)
val cubeGeomPaths2 = cube2.getGeomPaths(camera)


// Plot the lines on the screen
plot +
        cubeGeomPaths[0] + cubeGeomPaths[1] + cubeGeomPaths[2] + cubeGeomPaths[3] + cubeGeomPaths[4] + cubeGeomPaths[5] + cubeGeomPaths[6] + cubeGeomPaths[7] + cubeGeomPaths[8] + cubeGeomPaths[9] + cubeGeomPaths[10] + cubeGeomPaths[11] +
        cubeGeomPaths2[0] + cubeGeomPaths2[1] + cubeGeomPaths2[2] + cubeGeomPaths2[3] + cubeGeomPaths2[4] + cubeGeomPaths2[5] + cubeGeomPaths2[6] + cubeGeomPaths2[7] + cubeGeomPaths2[8] + cubeGeomPaths2[9] + cubeGeomPaths2[10] + cubeGeomPaths2[11]




## Explanation
The graph helps to explain the concept.
![Setup](https://raw.githubusercontent.com/Pascal1414/Renderer3d/main/setup.svg)

Within the setup are a Screen, a Camera, and Two points. The objective now lies in determining the positions of these points on the screen, namely Contact Point A and Contact Point B.

### Example  
To streamline the issue, let's narrow our focus to Point A and solely consider its position along the x-axis on the screen. For a visual representation, envision observing it from above, akin to a Top View.

![Top View](https://raw.githubusercontent.com/Pascal1414/Renderer3d/main/topView.svg)

We can then establish the subsequent formula to ascertain the x-coordinate of the point on the screen:

$$
xProjected = \frac{focalLength * xOffset}{focalLength + zDistance}
$$

The same thing needs to be done for the y position of the point and there we have the screen coordinates of the point A. That needs to be done for all the points that we want to display. Now we can draw lines between the points to display the edges. And thats how a 3d shape can be rendered on a 2d screen.